## Échantillonnage des données avec une fenêtre glissante


L'étape suivante pour créer les embeddings destinés au LLM consiste à générer les couples entrée-cible nécessaires à son entraînement. À quoi ressemblent ces couples ? Comme nous l'avons déjà vu, les LLM sont préentraînés en prédisant le mot suivant d'un texte.


In [10]:
import tiktoken
from torch.utils.data import DataLoader
from gptlight.datasets import fetch_verdict_text
from gptlight.utils.data import GPTDataset

In [11]:
tokenizer = tiktoken.get_encoding("gpt2")
raw_text = fetch_verdict_text()

In [13]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDataset(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

- `drop_last=True` supprime le dernier batch s'il est plus petit que `batch_size`, afin d'éviter des sauts de perte pendant l'entraînement.
- `num_workers` : nombre de processus CPU utilisés pour le prétraitement.
- `max_length` : longueur des séquences (fenêtres) envoyées au modèle ; par exemple, avec `max_length = 128`, chaque exemple contient 128 tokens.
- `stride` : décalage appliqué dans la liste de tokens entre deux fenêtres successives.


In [28]:
dataloader = create_dataloader_v1(
    raw_text, 
    batch_size=1, 
    max_length=4, 
    stride=1, 
    shuffle=False
)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [29]:
i = 1
for batch in data_iter:
    print(batch)
    i+=1
    if i >=10:
        break

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]
[tensor([[2885, 1464, 1807, 3619]]), tensor([[1464, 1807, 3619,  402]])]
[tensor([[1464, 1807, 3619,  402]]), tensor([[1807, 3619,  402,  271]])]
[tensor([[1807, 3619,  402,  271]]), tensor([[ 3619,   402,   271, 10899]])]
[tensor([[ 3619,   402,   271, 10899]]), tensor([[  402,   271, 10899,  2138]])]
[tensor([[  402,   271, 10899,  2138]]), tensor([[  271, 10899,  2138,   257]])]
[tensor([[  271, 10899,  2138,   257]]), tensor([[10899,  2138,   257,  7026]])]
[tensor([[10899,  2138,   257,  7026]]), tensor([[ 2138,   257,  7026, 15632]])]
[tensor([[ 2138,   257,  7026, 15632]]), tensor([[  257,  7026, 15632,   438]])]


In [31]:
dataloader = create_dataloader_v1(
    raw_text, 
    batch_size=8, 
    max_length=4, 
    stride=1, 
    shuffle=False
)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[10899,  2138,   257,  7026],
        [ 2138,   257,  7026, 15632],
        [  257,  7026, 15632,   438],
        [ 7026, 15632,   438,  2016],
        [15632,   438,  2016,   257],
        [  438,  2016,   257,   922],
        [ 2016,   257,   922,  5891],
        [  257,   922,  5891,  1576]])

Targets:
 tensor([[ 2138,   257,  7026, 15632],
        [  257,  7026, 15632,   438],
        [ 7026, 15632,   438,  2016],
        [15632,   438,  2016,   257],
        [  438,  2016,   257,   922],
        [ 2016,   257,   922,  5891],
        [  257,   922,  5891,  1576],
        [  922,  5891,  1576,   438]])
